## PIX - Phuture Infrastructure Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "infrastructure"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens]
    )
liquidity

Note: loom-network-new, marketcap data available only for 12 < 180 days
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
0x44c784266cf024a60e8acf2427b9857ace194c5d avalanche
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f optimistic-ethereum
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f base
0x8b1f4432f943c465a973fedc6d7aa50fc96f1f65 fantom
0xf3c091ed43de9c270593445163a41a876a0bb3dd arbitrum-one
0x319f865b287fcc10b30d8ce6144e8b6d1b476999 arbitrum-one
0x6b289cceaa8639e3831095d75a3e43520fabf552 avalanche
0xec6adef5e1006bb305bb1975333e8fc4071295bf optimistic-ethereum
0x8413041a7702603d9d991f2c4add29e4e8a241f8 optimistic-ethereum
0xf44ff799ea2bbfec96f9a50498209aac3c2b3b8b avalanche
0x11bbf12363dc8375b78d2719395d505f52a02f68 fantom
0x11bbf12363dc8375b78d2719395d505f52a02f68 arbitrum-one


,spot price,delivery price,slippage,blockchain
id,,,,
lido-dao,0.405911,0.406175,0.000651,ethereum
chainlink,0.059615,0.059601,-0.000234,arbitrum-one
rocket-pool,0.032189,0.031986,-0.006318,ethereum
fantom,0.339451,0.334092,-0.015786,fantom
axelar,1.264276,1.232017,-0.025516,ethereum
the-graph,5.793528,5.564921,-0.039459,ethereum
radicle,0.580598,0.554184,-0.045495,ethereum
altura,13.786502,12.273997,-0.109709,binance-smart-chain
route,0.245307,0.210827,-0.140558,polygon-pos


## Show Results

In [4]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
chainlink,Chainlink,9.336602e+09,0.300000,77,0xf97f4df75117a78c1a5a0dbb814af92458539fb4,arbitrum-one
lido-dao,Lido DAO,2.171185e+09,0.257743,66,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
the-graph,The Graph,1.634776e+09,0.194065,50,0xc944e90c64b2c07662a292be6244bdf05cda44a7,ethereum
fantom,Fantom,1.080667e+09,0.128287,33,FTM,fantom
rocket-pool,Rocket Pool,6.195978e+08,0.073553,18,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
axelar,Axelar,3.904693e+08,0.046353,11,0x467719ad09025fcc6cf6f8311755809d45a5e5f3,ethereum
